In [27]:
import pandas as pd
import numpy as np


bitcoin = pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv')
print(bitcoin.shape)
bitcoin_set = bitcoin.dropna() #dropping the nan values
print(bitcoin_set.head(5))
print(bitcoin_set.shape)

(647618, 8)
       Timestamp  Open  High  ...  Volume_(BTC)  Volume_(Currency)  Weighted_Price
0     1325317920  4.39  4.39  ...      0.455581           2.000000        4.390000
478   1325346600  4.39  4.39  ...     48.000000         210.720000        4.390000
547   1325350740  4.50  4.57  ...     37.862297         171.380338        4.526411
548   1325350800  4.58  4.58  ...      9.000000          41.220000        4.580000
1224  1325391360  4.58  4.58  ...      1.502000           6.879160        4.580000

[5 rows x 8 columns]
(47859, 8)


Discovering the dataset, its shape


In [31]:
bitcoin_set.drop(columns=['Low','High', 'Open', 'Close'], axis=0)
#dropping the unneeded features

,Timestamp,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,0.455581,2.000000,4.390000
478,1325346600,48.000000,210.720000,4.390000
547,1325350740,37.862297,171.380338,4.526411
548,1325350800,9.000000,41.220000,4.580000
1224,1325391360,1.502000,6.879160,4.580000
...,...,...,...,...
647572,1364172240,0.082000,5.680960,69.280000
647586,1364173080,0.161729,11.214285,69.340000
647589,1364173260,7.500000,526.500000,70.200000
647592,1364173440,6.674074,468.520000,70.200000


In [32]:
import datetime
#chnaging unisx time to standard readable time

date_column = pd.to_datetime(bitcoin_set['Timestamp'], unit='s')
bitcoin_set.assign(date_column)

TypeError: ignored

# New Section